In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/usr/local/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import numpy as np
import matplotlib.pyplot as plt


from HOTS.Event import Event, SimpleAlphabet
from HOTS.STS import STS
from HOTS.Layer import ClusteringLayer, FilterNHBD
from HOTS.Monitor import DisplayImage, DisplayHisto
from HOTS.Tools import SaveObject, LoadObject, GenerateHistogram
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.Monitor import DisplaySurface2D
from HOTS.Tools import EuclidianNorm, BattachaNorm, NormalizedNorm

In [3]:
tau = 0.02
R = 2
nb_cluster = [4, 8, 16]
ImageSize = (32, 32)

This notebook show classification on Simple Alphabet database

# Load Clustering, Training and Testing data

In [4]:
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(
    NbTrainingData=36,
    NbTestingData=40,
    ClusteringData=[1, 32, 19, 22, 29],
    OutOnePolarity=False,
    verbose=0,
    Path="../data/alphabet_ExtractedStabilized.mat",
    LabelPath="../data/alphabet_label.pkl",
)

# Defining the Network 

In [5]:
InputFilter = FilterNHBD(threshold=15, neighbourhood=2)
L1 = ClusteringLayer(
    tau=tau,
    R=R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)
L2 = ClusteringLayer(
    tau=10 * tau,
    R=2 * R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)
L3 = ClusteringLayer(
    tau=10 * 10 * tau,
    R=2 * 2 * R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)
Net = Network([InputFilter, L1, L2, L3])

# Training the Cluster on 5 images

In [6]:
ClusterLayer, event_output = Net.TrainCluster(
    event=event_cl, NbClusterList=nb_cluster, to_record=False, NbCycle=1
)

Generation of SpatioTemporal Surface in ------ 6.89 s
Clustering SpatioTemporal Surface in ------ 11.78 s
Generation of SpatioTemporal Surface in ------ 4.06 s
Clustering SpatioTemporal Surface in ------ 11.35 s
Generation of SpatioTemporal Surface in ------ 11.58 s
Clustering SpatioTemporal Surface in ------ 42.56 s


# Running the Network on training samples

In [7]:
event_o_tr = Net.RunNetwork(
    event_tr, NbClusterList=ClusterLayer
)  ## Warning : seems to put pressure on the memory

Generation of SpatioTemporal Surface in ------ 29.11 s
Generation of SpatioTemporal Surface in ------ 34.47 s
Generation of SpatioTemporal Surface in ------ 69.45 s


# Runing the Network on testing samples

In [8]:
event_o_te = Net.RunNetwork(event_te, NbClusterList=ClusterLayer)

Generation of SpatioTemporal Surface in ------ 24.92 s
Generation of SpatioTemporal Surface in ------ 27.15 s
Generation of SpatioTemporal Surface in ------ 57.47 s


# Classifying

In [9]:
Classif0 = Classifier(
    event_o_tr, event_o_te, TrainingLabel=label_tr, GroundTruth=label_te
)
prediction, accuracy, method = Classif0.HistogramDistance(to_print=True)

battacha:85.0% ### normalized:82.5% ### euclidian:85.0% ### 
